In [113]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())



====== WebDriver manager ======
Could not get version for chromium with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Chromium\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Chromium\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Chromium\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Chromium\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Chromium" /v version }"
Current chromium version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN chromium
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Mitchell\.wdm\dr

### Navigate to the webpage and search 

In [114]:
url = 'https://www.amazon.ca/'
driver.get(url)

In [115]:
def get_url(search_term):
    """Generate a url form search term"""
    template = 'https://www.amazon.ca/s?k={}&i=popular&crid=1HNLDFKVIXLLG&sprefix={}%2Cpopular%2C86&ref=nb_sb_noss'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term, search_term)

In [116]:
url = get_url('vinyl record')
print(url)

https://www.amazon.ca/s?k=vinyl+record&i=popular&crid=1HNLDFKVIXLLG&sprefix=vinyl+record%2Cpopular%2C86&ref=nb_sb_noss


In [117]:
driver.get(url)

### Extract The Data

In [109]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
results = soup.find_all('div', {'data-component-type': 's-search-result'})
len(results)

22

In [110]:
titles = []
prices = []
ratings = []
review_counts = []

item = results[10]
search = item.h2.a
search.text
# title = search.text.strip()
# url = 'https://www.amazon.ca' + search.get('href')
# price = item.find('span', 'a-offscreen').text[1:]
# rating = item.i.text.split(" ")[0]
# review_count = item.find('span', class_='a-size-base s-underline-text').text

# if (item.find('a', class_='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-bold').text) == "LP Record":
title = search.text.strip()
url = 'https://www.amazon.ca' + search.get('href')
price = item.find('span', 'a-offscreen').text[1:]
rating = item.i.text.split(" ")[0]
review_count = item.find('span', class_='a-size-base s-underline-text').text

titles.append(title)
prices.append(price)
ratings.append(rating)
    


In [111]:
search.text

'Acrylic Turntable Mat - YellowLit - LP Slipmat '

### Create function to grab data from each item

### Cleaning data to get rid of errors

In [118]:
def get_data(item):
    
    search = item.h2.a
    search.text    
    title = search.text.strip()
    
    url = 'https://www.amazon.ca' + search.get('href')
    
    try:
        price = item.find('span', 'a-offscreen').text[1:]
    except AttributeError:
        return
    
    
    try:
        rating = item.i.text.split(" ")[0]
        review_count = item.find('span', class_='a-size-base s-underline-text').text
    except AttributeError:
        rating=''
        review_count=''

    result = (title, price, rating, review_count, url)

    return(result)

In [119]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = get_data(item)
    if record:
        records.append(get_data(item))

In [130]:
import pandas as pd
# for row in records:
#     print(row[1])
x = pd.DataFrame(records)
x

,0,1,2,3,4
0,The Very Best Vol.2 [Import] [Audio CD] Fairuz,11.65,5.0,8,https://www.amazon.ca/gp/slredirect/picassoRed...
1,Original Hits: New Country Love Songs,34.99,4.5,2,https://www.amazon.ca/gp/slredirect/picassoRed...
2,Unlimited Love (Deluxe Edition) (Vinyl),56.42,,,https://www.amazon.ca/Unlimited-Deluxe-Vinyl-C...
3,Diamonds (2LP Vinyl),44.17,4.8,"3,573",https://www.amazon.ca/Diamonds-2LP-Vinyl-Elton...
4,Abbey Road (50th Anniversary Vinyl Edition),37.98,4.8,"7,256",https://www.amazon.ca/Abbey-Road-50th-Annivers...
5,Back In Black(180 Gram Vinyl),33.98,4.8,"9,190",https://www.amazon.ca/Back-Black-180-Gram-Viny...
6,Ella Fitzgerald & Louis Armstrong (Vinyl),25.74,4.8,"1,869",https://www.amazon.ca/Ella-Fitzgerald-Louis-Ar...
7,Highway To Hell(180 Gram Vinyl),29.99,4.8,"4,288",https://www.amazon.ca/Highway-Hell-180-Gram-Vi...
8,30 (Amazon Exclusive White Vinyl),53.06,4.5,"14,005",https://www.amazon.ca/30-Amazon-Exclusive-Whit...
9,Harry Styles (Vinyl),34.99,4.8,"9,480",https://www.amazon.ca/Harry-Styles-Vinyl/dp/B0...


### Get Next Page

In [136]:
def get_url(search_term):
    """Generate a url form search term"""
    template = 'https://www.amazon.ca/s?k={}&i=popular&crid=1HNLDFKVIXLLG&sprefix={}%2Cpopular%2C86&ref=nb_sb_noss'
    search_term = search_term.replace(' ', '+')

    # add page number
    url = template.format(search_term, search_term)

    # add page query placeholder
    url += '&page={}'

    return url

get_url('vinyl record')

'https://www.amazon.ca/s?k=vinyl+record&i=popular&crid=1HNLDFKVIXLLG&sprefix=vinyl+record%2Cpopular%2C86&ref=nb_sb_noss&page={}'

In [159]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType

def get_url(search_term):
    """Generate a url form search term"""
    template = 'https://www.amazon.ca/s?k={}&i=popular&crid=1HNLDFKVIXLLG&sprefix={}%2Cpopular%2C86&ref=nb_sb_noss'
    search_term = search_term.replace(' ', '+')

    # add page number
    url = template.format(search_term, search_term)

    # add page query placeholder
    url += '&page={}'

    return url

# fucntion to extract data from item 
def get_data(item):
    
    search = item.h2.a
    search.text    
    title = search.text.strip()
    
    url = 'https://www.amazon.ca' + search.get('href')
    
    try:
        price = item.find('span', 'a-offscreen').text[1:]
    except AttributeError:
        return
    
    try:
        rating = item.i.text.split(" ")[0]
        review_count = item.find('span', class_='a-size-base s-underline-text').text
    except AttributeError:
        rating=''
        review_count=''

    result = (title, price, rating, review_count, url)

    return(result)

def main(search_term):
    driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())

    records = []
    url = get_url(search_term)

    for page in range(1, 5):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})

        for item in results:
            record = get_data(item)
            if record:
                records.append(record)

    driver.close()

    return records

In [160]:
x = main('vinyl records')



====== WebDriver manager ======
Could not get version for chromium with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Chromium\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Chromium\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Chromium\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Chromium\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Chromium" /v version }"
Current chromium version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN chromium
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Mitchell\.wdm\dr

In [161]:
import pandas as pd
# for row in records:
#     print(row[1])
x = pd.DataFrame(x)
x

,0,1,2,3,4
0,Open Road (Vinyl),25.99,4.8,67,https://www.amazon.ca/gp/slredirect/picassoRed...
1,Be (Essential Edition),35.96,4.8,"2,121",https://www.amazon.ca/gp/slredirect/picassoRed...
2,Abbey Road (50th Anniversary Vinyl Edition),38.13,4.8,"7,278",https://www.amazon.ca/Abbey-Road-50th-Annivers...
3,Led Zeppelin (Remastered) [180g Vinyl LP],33.94,4.8,"3,873",https://www.amazon.ca/Led-Zeppelin-Remastered-...
4,Diamonds (2LP Vinyl),44.67,4.8,"3,581",https://www.amazon.ca/Diamonds-2LP-Vinyl-Elton...
...,...,...,...,...,...
83,The Battle at Garden's Gate,10.00,4.7,"2,110",https://www.amazon.ca/Battle-at-Gardens-Gate/d...
84,Metallica (Remastered) [Lp],38.48,4.9,423,https://www.amazon.ca/Metallica-Remastered-Lp/...
85,Moondance (Vinyl),36.98,4.8,"1,888",https://www.amazon.ca/Moondance-Vinyl-Van-Morr...
86,V BESTLIFE Record Brush Record Cleaning Velvet...,18.08,4.4,3,https://www.amazon.ca/gp/slredirect/picassoRed...
